In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import specificity_score

In [3]:
# loading train data 
train_data = pd.read_csv('C:/Users/SURFACE/Desktop/Task 2/shap_code/selected_train_features_with_plof_wo_cluster.csv')
train_data.head()

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,...,f_1018,f_1019,f_1020,f_1021,f_1022,f_1023,f_1024,loss,label,plof_scores
0,0.147035,1.147716,0.107814,0.0,1.014606,0.0,0.0,0.122150,1.559529,0.000000,...,0.000000,0.000000,0.628776,0.0,0.000000,0.000000,0.0,0.000002,0,0.0
1,0.000000,0.758245,0.287061,0.0,0.960584,0.0,0.0,0.048487,1.853845,0.000000,...,0.000000,0.000000,0.944606,0.0,0.000000,0.000000,0.0,0.000002,0,0.0
2,0.000000,0.758245,0.287061,0.0,0.960584,0.0,0.0,0.048487,1.853845,0.000000,...,0.000000,0.000000,0.944606,0.0,0.000000,0.000000,0.0,0.000002,0,0.0
3,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.277865,...,0.505390,0.745533,0.000000,0.0,0.059781,0.123379,0.0,0.000002,1,0.0
4,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.183769,...,0.571389,0.794509,0.000000,0.0,0.000000,0.000000,0.0,0.000003,1,0.0


In [4]:
# loading train data 
test_data = pd.read_csv('selected_test_features_with_plof_wo_cluster.csv')
test_data.head()

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,...,f_1018,f_1019,f_1020,f_1021,f_1022,f_1023,f_1024,loss,label,plof_scores
0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.271627,...,0.622322,0.601037,0.000000,0.0,0.0,0.081586,0.0,0.000011,1,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.273056,...,0.572066,0.750851,0.000000,0.0,0.0,0.064522,0.0,0.000012,1,0.0
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.163542,...,0.384511,0.687811,0.000000,0.0,0.0,0.017738,0.0,0.000016,1,0.0
3,0.123537,1.370321,0.172961,0.0,1.361574,0.227110,0.04416,0.000000,1.063966,0.000000,...,0.000000,0.000000,0.720995,0.0,0.0,0.000000,0.0,0.000022,0,0.0
4,0.067586,1.144258,0.009396,0.0,0.948973,0.041225,0.00000,0.015311,1.122422,0.000000,...,0.000000,0.000000,0.525233,0.0,0.0,0.000000,0.0,0.000025,0,0.0


In [5]:
print(f"After implementing uncertainty mining and pLOF, total train samples: {train_data.shape[0]}")
print(f"After implementing uncertainty mining and pLOF, total test samples: {test_data.shape[0]}")

After implementing uncertainty mining and pLOF, total train samples: 11200
After implementing uncertainty mining and pLOF, total test samples: 2800


In [6]:
# droping loss columns for both train data and making labels 
train_x = pd.DataFrame(train_data.drop(['label', 'loss', 'plof_scores'], axis=1))
train_y = train_data['label']
print(f"shape of train features: {train_x.shape}")
print(f"shape of train labels: {train_y.shape}")

shape of train features: (11200, 1024)
shape of train labels: (11200,)


In [7]:
# droping loss columns for both train data and making labels 
test_x = pd.DataFrame(test_data.drop(['label', 'loss', 'plof_scores'], axis=1))
test_y = test_data['label']
print(f"shape of train features: {test_x.shape}")
print(f"shape of train labels: {test_y.shape}")

shape of train features: (2800, 1024)
shape of train labels: (2800,)


In [8]:
#perform with feature scaling and compare 
# k-fold validation
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv=10):
    '''Function to perform 5 Folds Cross-Validation
     Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
    _scoring = ['accuracy', 'precision', 'recall', 'f1']
    results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
    return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

In [9]:
# function to predict with a specific estimator and evaluation metrices

def model_prediction(ml_classifier, train_x, train_y, test_x, test_y):
    '''
    ml_classifier: estimator
    name of instantiated machile learning classifier

    train_x: train data 

    train_y: train labels

    test_x: test data 

    test_y = test labels
      '''
    # fit to the model
    ml_classifier.fit(train_x, train_y)

    # prediction for test data 
    prediction = ml_classifier.predict(test_x)

    # accuracy
    acc = metrics.accuracy_score(prediction, test_y)
    print(f"Accuracy: {acc}")

    # ROC-AUC
    roc_auc = roc_auc_score(test_y, prediction)
    print("ROC-AUC score is: ", round(roc_auc, 4))

    # Matthew's coefficent 
    mcc = matthews_corrcoef(test_y, prediction)
    print("The Matthews correlation coefficinet (MCC) is: ", round(mcc, 4))

    # specificity
    sp = specificity_score(test_y, prediction, average='weighted')
    print(f"Weighted specificity: {sp:.4f}")

    # classification reports 
    print(classification_report(test_y, prediction)) 
    return prediction

In [10]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.5, fit_intercept=True, intercept_scaling=1,
                            class_weight=None, random_state=42, solver='lbfgs', max_iter=500, multi_class='auto', 
                            verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

lr_model.fit(train_x, train_y)

LogisticRegression(C=0.5, max_iter=500, random_state=42)

In [12]:
import shap
# defining shap explainer 
explainer = shap.Explainer(lr_model.predict, train_x) 

shap_values = explainer(train_x, max_evals=2000)

Permutation explainer: 11201it [9:29:32,  3.05s/it]                              


In [13]:
print(f"shap of train shap values: {shap_values.shape}")

shap of train shap values: (11200, 1024)


In [14]:
train_x = pd.DataFrame(train_data.drop(['label', 'loss', 'plof_scores'], axis=1))

In [15]:
# storing feature importance for both train and test data 
# train feature importance 
feature_importance = pd.DataFrame({'name': train_x.columns, 'importance': shap_values.abs.sum(0).values})
feature_importance_train = feature_importance.sort_values(by='importance', ascending=False).reset_index(drop=True)
# save feature importance indexes
feature_importance_train.to_csv(r'train_importance_lr_clf.csv', index=False)
feature_importance = None
print('done...')

done...


In [16]:
# visualize 
feature_importance_train.head(10)

,name,importance
0,f_821,431.495
1,f_286,299.355
2,f_969,295.380
3,f_977,278.395
4,f_694,239.745
5,f_392,237.430
6,f_36,227.035
7,f_577,222.165
8,f_275,216.225
9,f_358,213.715


### Feature Selection and Classification Using Shaply Values

In [18]:
f_importance_data = pd.read_csv('train_importance_lr_clf.csv')
f_importance_data.head(5)

,name,importance
0,f_821,431.495
1,f_286,299.355
2,f_969,295.380
3,f_977,278.395
4,f_694,239.745


In [40]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

In [41]:
# making ensemble
# defining top-5 models for ensemble based on accuracy

base_estimator = LogisticRegression()
 

adaboost_model = AdaBoostClassifier(n_estimators=25, learning_rate=0.5, algorithm='SAMME.R', random_state=42, 
                                    base_estimator= base_estimator)

logistic_reg_model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.5, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=42, solver='sag', 
                                        max_iter=500, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, 
                                        l1_ratio=None)

rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt',
                                max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, 
                                n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, 
                                ccp_alpha=0.0, max_samples=None)

mlp_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.01, 
                        batch_size='auto', learning_rate='constant', learning_rate_init=0.1, power_t=0.5, 
                        max_iter=600, shuffle=True, random_state=42, tol=0.0001, verbose=False, 
                        warm_start=False, momentum=0.8, nesterovs_momentum=True, early_stopping=False, 
                        validation_fraction=0.1, beta_1=0.7, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10,
                        max_fun=15000)

knn_model = KNeighborsClassifier(n_neighbors=35, weights='distance', algorithm='kd_tree', 
                               leaf_size=50, p=2, metric='minkowski', metric_params=None, n_jobs=None)

In [42]:
# taking all five models into a list
models = list()
models.append(('adaboost', adaboost_model))
models.append(('lr', logistic_reg_model))
models.append(('rf', rf_model))
models.append(('mlp', mlp_model))
models.append(('knn', knn_model))

In [43]:
# function for MSE loss
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [44]:
def compute_weights(losses):
    rms = np.sqrt(np.mean(np.square(losses)))
    weights = [1 / (loss / rms) for loss in losses]
    return weights


In [45]:
# function to calculate loss
def ensemble_eval_loss(models, train_x, test_x, train_y, test_y):
    losses = list()
    for name, model in models:
        print("fitting \n", model, "\n....")
        model.fit(train_x, train_y)
        y_hat = model.predict(test_x)
        acc = metrics.accuracy_score(test_y, y_hat)
        loss = mse_loss(test_y, y_hat)
        print(f"MSE loss for {name}: {loss:.4f}")
        losses.append(loss)
    return losses

In [46]:
def select_features(train_x, test_x, num_of_feat):
    # making train and test set using specific number of features based on feature importance that 
    # we got from shapley values
    print(f"shape of train data before feature selection: {train_x.shape}")
    print(f"shape of test data before feature selection: {test_x.shape}")
    train_x = train_x[f_importance_data.head(num_of_feat)["name"]]
    test_x = test_x[f_importance_data.head(num_of_feat)["name"]]
    print(f"shape of train data after feature selection: {train_x.shape}")
    print(f"shape of test data after feature selection: {test_x.shape}")
    return train_x, test_x

In [47]:
def build_ensemble(models, train_x, test_x, train_y, test_y):
    '''
    all_loss: calculates loss of individual models in ensemble
    
    c_weights: calculates for individual models based on loss

    train_x: train data 

    train_y: train labels

    test_x: test data 

    test_y = test labels
    
      '''
    all_losses = ensemble_eval_loss(models, train_x, test_x, train_y, test_y)
    c_weights = compute_weights(all_losses)
    ensemble = VotingClassifier(estimators=models, voting='soft', weights=c_weights)
    ensemble.fit(train_x, train_y)
    y_hat = ensemble.predict(test_x)
    score = metrics.accuracy_score(test_y, y_hat)
    print('Weighted Avg Accuracy: %.3f' % (score*100))
    return ensemble

In [48]:
# function to predict with a specific estimator and evaluation metrices

def model_prediction_with_fs(model, train_x, train_y, test_x, test_y):
    '''
    ml_classifier: estimator
    name of instantiated machile learning classifier

    train_x: train data 

    train_y: train labels

    test_x: test data 

    test_y = test labels
    
      '''
    
    # fit to the model
#     ml_classifier.fit(train_x, train_y)

    # prediction for test data 
    prediction = model.predict(test_x)

    # accuracy
    acc = metrics.accuracy_score(prediction, test_y)
    print(f"Accuracy: {acc}")

    # ROC-AUC
    roc_auc = roc_auc_score(test_y, prediction)
    print("ROC-AUC score is: ", round(roc_auc, 3))

    # Matthew's coefficent 
    mcc = matthews_corrcoef(test_y, prediction)
    print("The Matthews correlation coefficinet (MCC) is: ", round(mcc, 3))

    # specificity
    sp = specificity_score(test_y, prediction, average='weighted')
    print(f"Weighted specificity: {sp:.3f}")

    # classification reports 
    print(classification_report(test_y, prediction)) 
    return prediction

In [49]:
def cross_validation(model, num_of_feat, _X, _y, _cv=10):
    '''Function to perform 5 Folds Cross-Validation
     Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
    # get selected features 
    _X = _X[f_importance_data.head(num_of_feat)["name"]]
    print(f"shape of train data after feature selection: {_X.shape}")
    
    _scoring = ['accuracy', 'precision', 'recall', 'f1']
    results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
    return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

#### 100 feartues

In [50]:
train_100, test_100 = select_features(train_x, test_x, 100)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 100)
shape of test data after feature selection: (2800, 100)


In [51]:
ensemble_100 = build_ensemble(models, train_100, test_100, train_y, test_y)

fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0868
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0821
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0879
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0875
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0904
Weighted Avg Accuracy: 91.536


In [52]:
ensemble_100

VotingClassifier(estimators=[('adaboost',
                              AdaBoostClassifier(base_estimator=LogisticRegression(),
                                                 learning_rate=0.5,
                                                 n_estimators=25,
                                                 random_state=42)),
                             ('lr',
                              LogisticRegression(C=0.5, max_iter=500,
                                                 random_state=42,
                                                 solver='sag')),
                             ('rf',
                              RandomForestClassifier(max_features='sqrt')),
                             ('mlp',
                              MLPClassifier(alpha=0.01, beta_1=0.7,
                                            learning_rate_init=0.1,
                                            max_iter=600, momentum=0.8,
                                            random_state=42)),
             

In [53]:
cross_validation(ensemble_100, 100, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 100)


{'Training Accuracy scores': array([0.9703373 , 0.97718254, 0.97748016, 0.9719246 , 0.97549603,
        0.97628968, 0.97549603, 0.97142857, 0.97470238, 0.97539683]),
 'Mean Training Accuracy': 97.45734126984127,
 'Training Precision scores': array([0.97495961, 0.97969849, 0.97816944, 0.97237237, 0.97769741,
        0.97696775, 0.97943134, 0.97121152, 0.98134253, 0.9744511 ]),
 'Mean Training Precision': 0.976630155429075,
 'Training Recall scores': array([0.96502099, 0.97421547, 0.97640944, 0.9710116 , 0.97281088,
        0.97520992, 0.9710116 , 0.97121152, 0.96741303, 0.9760096 ]),
 'Mean Training Recall': 0.9720324023099135,
 'Training F1 scores': array([0.96996484, 0.97694929, 0.97728864, 0.97169151, 0.97524802,
        0.97608804, 0.97520329, 0.97121152, 0.974328  , 0.97522972]),
 'Mean Training F1 Score': 0.9743202877013324,
 'Validation Accuracy scores': array([1.        , 0.99821429, 0.75803571, 0.89196429, 0.92142857,
        0.90892857, 0.91964286, 0.9125    , 0.91696429, 0.92

In [54]:
pred_100 = model_prediction_with_fs(ensemble_100, train_100, train_y, test_100, test_y)

Accuracy: 0.9153571428571429
ROC-AUC score is:  0.915
The Matthews correlation coefficinet (MCC) is:  0.831
Weighted specificity: 0.915
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      1423
           1       0.92      0.91      0.91      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 200 features 

In [55]:
train_200, test_200 = select_features(train_x, test_x, 200)
ensemble_200 = build_ensemble(models, train_200, test_200, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 200)
shape of test data after feature selection: (2800, 200)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0861
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0825
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0875
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0836
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0882
Weighted Avg Accuracy: 91.821


In [56]:
cross_validation(ensemble_200, 200, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 200)


{'Training Accuracy scores': array([0.97430556, 0.97529762, 0.97619048, 0.97559524, 0.975     ,
        0.97440476, 0.98253968, 0.98293651, 0.975     , 0.97400794]),
 'Mean Training Accuracy': 97.65277777777777,
 'Training Precision scores': array([0.97726358, 0.97069307, 0.98473127, 0.97866345, 0.98057467,
        0.9766881 , 0.98317982, 0.98396794, 0.97999192, 0.97975709]),
 'Mean Training Precision': 0.9795510888826433,
 'Training Recall scores': array([0.97081751, 0.97981211, 0.96701319, 0.9720112 , 0.96881248,
        0.97161136, 0.98160736, 0.98160736, 0.96941224, 0.96761295]),
 'Mean Training Recall': 0.9730317746976764,
 'Training F1 scores': array([0.97402988, 0.97523127, 0.97579181, 0.97532598, 0.97465809,
        0.97414311, 0.98239296, 0.98278623, 0.97467337, 0.97364715]),
 'Mean Training F1 Score': 0.9762679849041065,
 'Validation Accuracy scores': array([1.        , 0.99732143, 0.77767857, 0.89375   , 0.92232143,
        0.90982143, 0.92053571, 0.90982143, 0.92410714, 0.9

In [57]:
pred_200 = model_prediction_with_fs(ensemble_200, train_200, train_y, test_200, test_y)

Accuracy: 0.9182142857142858
ROC-AUC score is:  0.918
The Matthews correlation coefficinet (MCC) is:  0.837
Weighted specificity: 0.918
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1423
           1       0.93      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 300 features

In [59]:
train_300, test_300 = select_features(train_x, test_x, 300)
ensemble_300 = build_ensemble(models, train_300, test_300, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 300)
shape of test data after feature selection: (2800, 300)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0857
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0814
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0832
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.1007
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0879
Weighted Avg Accuracy: 91.929


In [60]:
cross_validation(ensemble_300, 300, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 300)


{'Training Accuracy scores': array([0.97480159, 0.9796627 , 0.98194444, 0.97926587, 0.98144841,
        0.9780754 , 0.97738095, 0.97837302, 0.98125   , 0.97956349]),
 'Mean Training Accuracy': 97.91765873015873,
 'Training Precision scores': array([0.97998787, 0.97865124, 0.98706548, 0.98035678, 0.98236826,
        0.98577525, 0.98125   , 0.98421053, 0.982942  , 0.98210696]),
 'Mean Training Precision': 0.9824714371501375,
 'Training Recall scores': array([0.96901859, 0.98041175, 0.97640944, 0.97780888, 0.98020792,
        0.96981208, 0.9730108 , 0.9720112 , 0.97920832, 0.97660936]),
 'Mean Training Recall': 0.9754508310607397,
 'Training F1 scores': array([0.97447236, 0.9795307 , 0.98170854, 0.97908117, 0.9812869 ,
        0.97772851, 0.97711303, 0.97807282, 0.98107161, 0.97935044]),
 'Mean Training F1 Score': 0.9789416092227906,
 'Validation Accuracy scores': array([1.        , 0.99642857, 0.76875   , 0.89285714, 0.92410714,
        0.90803571, 0.91875   , 0.90892857, 0.92232143, 0.9

In [61]:
pred_300 = model_prediction_with_fs(ensemble_300, train_300, train_y, test_300, test_y)

Accuracy: 0.9192857142857143
ROC-AUC score is:  0.919
The Matthews correlation coefficinet (MCC) is:  0.839
Weighted specificity: 0.919
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1423
           1       0.92      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 400 features 

In [62]:
train_400, test_400 = select_features(train_x, test_x, 400)
ensemble_400 = build_ensemble(models, train_400, test_400, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 400)
shape of test data after feature selection: (2800, 400)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0854
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0804
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0857
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0832
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0861
Weighted Avg Accuracy: 91.821


In [63]:
cross_validation(ensemble_400, 400, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 400)


{'Training Accuracy scores': array([0.97480159, 0.97619048, 0.97847222, 0.97222222, 0.97867063,
        0.97400794, 0.97539683, 0.97321429, 0.9781746 , 0.9734127 ]),
 'Mean Training Accuracy': 97.54563492063491,
 'Training Precision scores': array([0.97652017, 0.97888598, 0.98856443, 0.97050618, 0.98757384,
        0.98446443, 0.97769293, 0.97875354, 0.98069964, 0.9783751 ]),
 'Mean Training Precision': 0.980203623982483,
 'Training Recall scores': array([0.97261643, 0.97301619, 0.96781287, 0.97361056, 0.96921232,
        0.96281487, 0.97261096, 0.96701319, 0.97520992, 0.96781287]),
 'Mean Training Recall': 0.9701730181403352,
 'Training F1 scores': array([0.97456439, 0.97594226, 0.97807859, 0.97205589, 0.97830693,
        0.9735193 , 0.97514532, 0.97284795, 0.97794707, 0.97306533]),
 'Mean Training F1 Score': 0.9751473037738982,
 'Validation Accuracy scores': array([1.        , 0.99821429, 0.77053571, 0.89464286, 0.92053571,
        0.90803571, 0.91785714, 0.91160714, 0.91964286, 0.92

In [64]:
pred_400 = model_prediction_with_fs(ensemble_400, train_400, train_y, test_400, test_y)

Accuracy: 0.9182142857142858
ROC-AUC score is:  0.918
The Matthews correlation coefficinet (MCC) is:  0.836
Weighted specificity: 0.918
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1423
           1       0.92      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 500 features 

In [65]:
train_500, test_500 = select_features(train_x, test_x, 500)
ensemble_500 = build_ensemble(models, train_500, test_500, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 500)
shape of test data after feature selection: (2800, 500)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0871
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0814
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0850
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0857
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0850
Weighted Avg Accuracy: 91.786


In [66]:
cross_validation(ensemble_500, 500, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 500)


{'Training Accuracy scores': array([0.97539683, 0.97490079, 0.98005952, 0.97579365, 0.97628968,
        0.97549603, 0.97331349, 0.97668651, 0.97728175, 0.97698413]),
 'Mean Training Accuracy': 97.6220238095238,
 'Training Precision scores': array([0.9805943 , 0.97424121, 0.98309519, 0.97637165, 0.97907866,
        0.97962477, 0.97606115, 0.98083518, 0.98339072, 0.98377282]),
 'Mean Training Precision': 0.9797065658892663,
 'Training Recall scores': array([0.96961823, 0.97521487, 0.97660936, 0.97481008, 0.9730108 ,
        0.97081168, 0.970012  , 0.9720112 , 0.97061176, 0.96961216]),
 'Mean Training Recall': 0.9722322104538156,
 'Training F1 scores': array([0.97507538, 0.9747278 , 0.97984154, 0.97559024, 0.9760353 ,
        0.97519831, 0.97302717, 0.97640325, 0.97695945, 0.97664116]),
 'Mean Training F1 Score': 0.9759499600503048,
 'Validation Accuracy scores': array([1.        , 0.99821429, 0.77410714, 0.89107143, 0.92232143,
        0.90714286, 0.91517857, 0.91071429, 0.91785714, 0.92

In [67]:
pred_500 = model_prediction_with_fs(ensemble_500, train_500, train_y, test_500, test_y)

Accuracy: 0.9178571428571428
ROC-AUC score is:  0.918
The Matthews correlation coefficinet (MCC) is:  0.836
Weighted specificity: 0.917
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1423
           1       0.93      0.90      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 600 features 

In [68]:
train_600, test_600 = select_features(train_x, test_x, 600)
ensemble_600 = build_ensemble(models, train_600, test_600, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 600)
shape of test data after feature selection: (2800, 600)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0879
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0804
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0871
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0861
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0864
Weighted Avg Accuracy: 91.607


In [69]:
cross_validation(ensemble_600, 600, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 600)


{'Training Accuracy scores': array([0.97390873, 0.98045635, 0.9796627 , 0.97857143, 0.98055556,
        0.97668651, 0.97519841, 0.97619048, 0.97837302, 0.97886905]),
 'Mean Training Accuracy': 97.78472222222221,
 'Training Precision scores': array([0.97743755, 0.97983227, 0.98230444, 0.98362975, 0.98565077,
        0.97890295, 0.97482014, 0.98081583, 0.98090068, 0.9795714 ]),
 'Mean Training Precision': 0.9803865793084459,
 'Training Recall scores': array([0.96981811, 0.98081151, 0.97660936, 0.9730108 , 0.97501   ,
        0.9740104 , 0.97520992, 0.9710116 , 0.97540984, 0.97780888]),
 'Mean Training Recall': 0.9748710389919587,
 'Training F1 scores': array([0.97361292, 0.98032165, 0.97944862, 0.97829146, 0.98030151,
        0.97645055, 0.97501499, 0.97588909, 0.97814755, 0.97868934]),
 'Mean Training F1 Score': 0.9776167680979618,
 'Validation Accuracy scores': array([1.        , 0.99732143, 0.76517857, 0.89642857, 0.92232143,
        0.91071429, 0.91785714, 0.90892857, 0.91964286, 0.9

In [70]:
pred_600 = model_prediction_with_fs(ensemble_600, train_600, train_y, test_600, test_y)

Accuracy: 0.9160714285714285
ROC-AUC score is:  0.916
The Matthews correlation coefficinet (MCC) is:  0.832
Weighted specificity: 0.915
              precision    recall  f1-score   support

           0       0.90      0.93      0.92      1423
           1       0.93      0.90      0.91      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 700 features 

In [71]:
train_700, test_700 = select_features(train_x, test_x, 700)
ensemble_700 = build_ensemble(models, train_700, test_700, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 700)
shape of test data after feature selection: (2800, 700)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0879
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0800
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0839
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0829
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0857
Weighted Avg Accuracy: 91.929


In [72]:
cross_validation(ensemble_700, 700, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 700)


{'Training Accuracy scores': array([0.97162698, 0.97867063, 0.97886905, 0.97589286, 0.97371032,
        0.97083333, 0.9764881 , 0.97450397, 0.97748016, 0.97549603]),
 'Mean Training Accuracy': 97.53571428571428,
 'Training Precision scores': array([0.97445182, 0.97368421, 0.98111312, 0.97504492, 0.97993921,
        0.97136564, 0.98375635, 0.97919612, 0.97855282, 0.97712222]),
 'Mean Training Precision': 0.9774226417979349,
 'Training Recall scores': array([0.96821907, 0.98360983, 0.97620952, 0.97640944, 0.96681327,
        0.96981208, 0.96861255, 0.96921232, 0.9760096 , 0.97341064]),
 'Mean Training Recall': 0.972831830689671,
 'Training F1 scores': array([0.97132545, 0.97862186, 0.97865518, 0.9757267 , 0.97333199,
        0.97058824, 0.97612572, 0.97417864, 0.97727955, 0.97526289]),
 'Mean Training F1 Score': 0.9751096208207191,
 'Validation Accuracy scores': array([1.        , 0.99732143, 0.76607143, 0.89464286, 0.92589286,
        0.91071429, 0.91607143, 0.90982143, 0.92232143, 0.92

In [73]:
pred_700 = model_prediction_with_fs(ensemble_700, train_700, train_y, test_700, test_y)

Accuracy: 0.9192857142857143
ROC-AUC score is:  0.919
The Matthews correlation coefficinet (MCC) is:  0.839
Weighted specificity: 0.919
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1423
           1       0.93      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 800 features 

In [74]:
train_800, test_800 = select_features(train_x, test_x, 800)
ensemble_800 = build_ensemble(models, train_800, test_800, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 800)
shape of test data after feature selection: (2800, 800)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0864
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0796
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0857
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0846
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0846
Weighted Avg Accuracy: 91.821


In [75]:
cross_validation(ensemble_800, 800, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 800)


{'Training Accuracy scores': array([0.96865079, 0.97261905, 0.97906746, 0.97559524, 0.97549603,
        0.97628968, 0.97847222, 0.97986111, 0.97996032, 0.97757937]),
 'Mean Training Accuracy': 97.6359126984127,
 'Training Precision scores': array([0.96879376, 0.97241655, 0.98247734, 0.9756    , 0.98451192,
        0.97639528, 0.97897898, 0.986418  , 0.98      , 0.97664671]),
 'Mean Training Precision': 0.9782238537793798,
 'Training Recall scores': array([0.96801919, 0.97241655, 0.97520992, 0.97520992, 0.96581367,
        0.97580968, 0.97760896, 0.97281088, 0.97960816, 0.97820872]),
 'Mean Training Recall': 0.9740715626601648,
 'Training F1 scores': array([0.96840632, 0.97241655, 0.97883014, 0.97540492, 0.97507317,
        0.97610239, 0.97829349, 0.97956719, 0.97980404, 0.97742709]),
 'Mean Training F1 Score': 0.976132528429973,
 'Validation Accuracy scores': array([1.        , 0.99910714, 0.77321429, 0.89196429, 0.92142857,
        0.90892857, 0.91785714, 0.90803571, 0.92232143, 0.925

In [76]:
pred_800 = model_prediction_with_fs(ensemble_800, train_800, train_y, test_800, test_y)

Accuracy: 0.9182142857142858
ROC-AUC score is:  0.918
The Matthews correlation coefficinet (MCC) is:  0.836
Weighted specificity: 0.918
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1423
           1       0.92      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800



#### 900 features

In [77]:
train_900, test_900 = select_features(train_x, test_x, 900)
ensemble_900 = build_ensemble(models, train_900, test_900, train_y, test_y)

shape of train data before feature selection: (11200, 1024)
shape of test data before feature selection: (2800, 1024)
shape of train data after feature selection: (11200, 900)
shape of test data after feature selection: (2800, 900)
fitting 
 AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=0.5,
                   n_estimators=25, random_state=42) 
....
MSE loss for adaboost: 0.0868
fitting 
 LogisticRegression(C=0.5, max_iter=500, random_state=42, solver='sag') 
....
MSE loss for lr: 0.0793
fitting 
 RandomForestClassifier(max_features='sqrt') 
....
MSE loss for rf: 0.0850
fitting 
 MLPClassifier(alpha=0.01, beta_1=0.7, learning_rate_init=0.1, max_iter=600,
              momentum=0.8, random_state=42) 
....
MSE loss for mlp: 0.0829
fitting 
 KNeighborsClassifier(algorithm='kd_tree', leaf_size=50, n_neighbors=35,
                     weights='distance') 
....
MSE loss for knn: 0.0843
Weighted Avg Accuracy: 91.893


In [78]:
cross_validation(ensemble_900, 900, train_x, train_y, _cv=10)

shape of train data after feature selection: (11200, 900)


{'Training Accuracy scores': array([0.97311508, 0.97986111, 0.97857143, 0.98105159, 0.98005952,
        0.97698413, 0.9781746 , 0.98085317, 0.97886905, 0.97698413]),
 'Mean Training Accuracy': 97.84523809523809,
 'Training Precision scores': array([0.9824633 , 0.97961631, 0.98480551, 0.97548923, 0.97695212,
        0.98240291, 0.97973515, 0.97529156, 0.9846185 , 0.97795591]),
 'Mean Training Precision': 0.9799330490722037,
 'Training Recall scores': array([0.96302219, 0.97981211, 0.97181128, 0.98660536, 0.9830068 ,
        0.9710116 , 0.97620952, 0.98640544, 0.97261096, 0.97560976]),
 'Mean Training Recall': 0.976610499114366,
 'Training F1 scores': array([0.9726456 , 0.9797142 , 0.97826524, 0.9810158 , 0.9799701 ,
        0.97667404, 0.97796916, 0.98081702, 0.97857789, 0.97678143]),
 'Mean Training F1 Score': 0.9782430488897701,
 'Validation Accuracy scores': array([1.        , 0.99821429, 0.77321429, 0.89553571, 0.92053571,
        0.90714286, 0.91875   , 0.91160714, 0.92053571, 0.92

In [79]:
pred_900 = model_prediction_with_fs(ensemble_900, train_900, train_y, test_900, test_y)

Accuracy: 0.9189285714285714
ROC-AUC score is:  0.919
The Matthews correlation coefficinet (MCC) is:  0.838
Weighted specificity: 0.918
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1423
           1       0.93      0.91      0.92      1377

    accuracy                           0.92      2800
   macro avg       0.92      0.92      0.92      2800
weighted avg       0.92      0.92      0.92      2800

